# ~~~~~ Day 16 ~~~~~ 
## Part 1

In [1]:
def char_to_bin_string(h):
    return bin(int('1'+h, 16))[3:]

def bin_to_dec(b):
    d = 0
    for b1 in b:
        d = d << 1 | int(b1)
    return d

In [2]:
with open('d16.txt', 'r') as f:
    content = f.readlines()

In [3]:
def parse_it(data, offset=0, subpackets=0):
    pos = 0
    decoded = []
    while pos<len(data)-6:
        ver = bin_to_dec(data[pos:pos+3])
        pos += 3
        ptype = bin_to_dec(data[pos:pos+3])
        pos += 3
        if ptype == 4: #  Literal
            payloads = []
            while True:
                more = int(data[pos])
                pos += 1
                payload = data[pos:pos+4]
                pos += 4
                payloads.append(payload)
                if not more: break
            decoded_payload = bin_to_dec(''.join(payloads))
            p = {'version': ver, 'type': ptype, 'payload': decoded_payload}
            print(f'Literal packet version {ver}: {p}')
            decoded.append(p)
        else: 
            length_type = int(data[pos])
            pos += 1
            pkt_length = 0
            if length_type == 0:
                pkt_length = bin_to_dec(data[pos:pos+15])
                pos += 15
            else:
                pkt_length = bin_to_dec(data[pos:pos+11])
                pos += 11
            subpos, subdec = parse_it(
                data[pos:] if length_type == 1 else data[pos:pos+pkt_length],
                subpackets=pkt_length if length_type == 1 else 0)
            pos += subpos
            if len(subdec)>0:
                print(f'Operator packet version {ver} type {ptype}, length {pkt_length}')
                decoded.append({'version': ver, 'type': ptype, 'length_type': length_type, 'length': pkt_length, 'subpackets': subdec})
        if subpackets > 0 and len(decoded) >= subpackets:
            return (pos, decoded)
    return (pos, decoded)

def get_ver(d):
    v = []
    for i in d:
        v.append(i['version'])
        if 'subpackets' in i.keys():
            v.extend(get_ver(i['subpackets']))
    return v

decoded = []
for c in [c.strip('\n') for c in content]:
    print(f'-------- Parsing {c}')
    n, dec = parse_it(''.join(char_to_bin_string(c)))
    decoded.append((c, dec))
    print(dec)
    print(f'Version sum: {sum(get_ver(dec))}')
    print('\n')

-------- Parsing A052E04CFD9DC0249694F0A11EA2044E200E9266766AB004A525F86FFCDF4B25DFC401A20043A11C61838600FC678D51B8C0198910EA1200010B3EEA40246C974EF003331006619C26844200D414859049402D9CDA64BDEF3C4E623331FBCCA3E4DFBBFC79E4004DE96FC3B1EC6DE4298D5A1C8F98E45266745B382040191D0034539682F4E5A0B527FEB018029277C88E0039937D8ACCC6256092004165D36586CC013A008625A2D7394A5B1DE16C0E3004A8035200043220C5B838200EC4B8E315A6CEE6F3C3B9FFB8100994200CC59837108401989D056280803F1EA3C41130047003530004323DC3C860200EC4182F1CA7E452C01744A0A4FF6BBAE6A533BFCD1967A26E20124BE1920A4A6A613315511007A4A32BE9AE6B5CAD19E56BA1430053803341007E24C168A6200D46384318A6AAC8401907003EF2F7D70265EFAE04CCAB3801727C9EC94802AF92F493A8012D9EABB48BA3805D1B65756559231917B93A4B4B46009C91F600481254AF67A845BA56610400414E3090055525E849BE8010397439746400BC255EE5362136F72B4A4A7B721004A510A7370CCB37C2BA0010D3038600BE802937A429BD20C90CCC564EC40144E80213E2B3E2F3D9D6DB0803F2B005A731DC6C524A16B5F1C1D98EE006339009AB401AB0803108A12C2A00043A134228AB2DBDA

## Part 2

In [4]:
int([3]>[2])

1

In [5]:
import numpy as np
def iprint(i, s):
    print(f'{"".join([" " for i in range(0, i)])}{s}')
    
def recurse_pkt(i, p, op=0):
    pt = p['type']
    if pt == 4: 
        l = p['payload']
        iprint(i, f'Literal {l}')
        return l
    op += 1
    iprint(i, f'Start OP {op}')
    a = [recurse_pkt(i+1, s, op) for s in p['subpackets']]
    values = np.asarray(a)
    if pt == 0: 
        r = np.sum(values)
        iprint(i, f'OP {op} sum {values.transpose()} = {r}')
        return r
    if pt == 1:
        r = np.prod(values)
        iprint(i, f'OP {op} prod {values.transpose()} = {r}')
        return r
    if pt == 2:
        r = np.min(values)
        iprint(i, f'OP {op} min {values.transpose()} = {r}')
        return r
    if pt == 3:
        r = np.max(values)
        iprint(i, f'OP {op} max {values.transpose()} = {r}')
        return r
    if pt == 5:
        r = int(values[0]>values[1])
        iprint(i, f'OP {op}  >  {values.transpose()} = {r}')
        return r
    if pt == 6:
        iprint(i, f'OP {op} <  {values.transpose()} {len(p["subpackets"])}')
        r = int(values[0]<values[1])
        iprint(i, f'OP {op}  <  {values.transpose()} = {r}')
        return r
    if pt == 7: 
        r = int(values[0]==values[1])
        iprint(i, f'OP {op}  == {values.transpose()} = {r}')
        return r
    
for c, d in decoded:
    print(f'{c} -> {recurse_pkt(0, d[0])}')

Start OP 1
 Literal 2556
 Start OP 2
  Literal 402314
  Literal 15
  Literal 8
  Literal 9
 OP 2 max [402314     15      8      9] = 402314
 Start OP 2
  Literal 37741
  Start OP 3
   Literal 287506942933
   Literal 127
  OP 3  >  [287506942933          127] = 1
 OP 2 prod [37741     1] = 37741
 Start OP 2
  Start OP 3
   Literal 4
   Literal 12
   Literal 3
  OP 3 prod [ 4 12  3] = 144
  Start OP 3
   Literal 12
   Literal 13
   Literal 13
  OP 3 prod [12 13 13] = 2028
  Start OP 3
   Literal 9
   Literal 7
   Literal 4
  OP 3 prod [9 7 4] = 252
 OP 2 sum [ 144 2028  252] = 2424
 Start OP 2
  Literal 254
 OP 2 sum [254] = 254
 Start OP 2
  Literal 727
  Start OP 3
   Start OP 4
    Literal 6
    Literal 4
    Literal 4
   OP 4 sum [6 4 4] = 14
   Start OP 4
    Literal 2
    Literal 5
    Literal 2
   OP 4 sum [2 5 2] = 9
  OP 3 <  [14  9] 2
  OP 3  <  [14  9] = 0
 OP 2 prod [727   0] = 0
 Start OP 2
  Literal 3257
  Literal 2004058930
  Literal 37375
  Literal 79
  Literal 200145135
